### Study of effect of national factors on home prices in US

**Task -** Using publically available data for the national factors that impact supply and demand of homes in US, build a model to study the effect of these variables on home prices.

**Approach -** The following variables are chosen for the study-
- Unemployment Rate
- Per Capita GDP
- Median Household Income
- Construction Prices
- CPI 
- Interest Rates
- Number of new houses supplied
- Working Population
- Urban Population
- Percentage of population above 65
- Housing subsidies
- Number of Households

As a proxy to the home prices, S&P CASE-SHILLER Index is used. 

Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed and combined to create a datset. Data for different variables had different frequencies. So, to combine the data, necessary interpolations are made.

Linear Regression is used as most of the variables have high correlation with the target variable.

In [453]:
# importing libraries
import numpy as np
import pandas as pd

In [427]:
#Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("CASESHILLER.csv")

#Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

#Selecting data post JUNE 2001
mask = df_CS["DATE"] >= "2001-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print(df_CS.shape)
df_CS.tail()

(252, 4)


,DATE,CSUSHPISA,Year,Month
247,2022-02-01,290.371,2022,2
248,2022-03-01,296.362,2022,3
249,2022-04-01,301.409,2022,4
250,2022-05-01,305.179,2022,5
251,2022-06-01,306.181,2022,6


In [428]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
df_unemp.drop([252,253], inplace = True)
print(df_unemp.shape)
df_unemp.tail()

(252, 2)


,DATE,UNRATE
247,2022-02-01,3.8
248,2022-03-01,3.6
249,2022-04-01,3.6
250,2022-05-01,3.6
251,2022-06-01,3.6


In [429]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("gdp_per_capita.csv", names = ["DATE", "Per_Capita_GDP"], skiprows = 1)
print(df_pcgdp.shape)
df_pcgdp.tail()

(84, 2)


,DATE,Per_Capita_GDP
79,2021-04-01,58335.0
80,2021-07-01,58619.0
81,2021-10-01,59553.0
82,2022-01-01,59288.0
83,2022-04-01,59168.0


The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.

In [431]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("FEDFUNDS (1).csv").drop([252,253])
print(df_Fed_rate.shape)
df_Fed_rate.tail()

(252, 2)


,DATE,FEDFUNDS
247,2022-02-01,0.08
248,2022-03-01,0.20
249,2022-04-01,0.33
250,2022-05-01,0.77
251,2022-06-01,1.21


In [430]:
# Reading Per Capita GDP Data into a dataframe
df_cons_price_index = pd.read_csv("Construction_Price.csv", names = ["DATE", "Cons_Materials"], skiprows = 1)
df_cons_price_index.drop([252], inplace = True)
print(df_cons_price_index.shape)
df_cons_price_index.tail()

(252, 2)


,DATE,Cons_Materials
247,2022-02-01,343.583
248,2022-03-01,345.852
249,2022-04-01,343.730
250,2022-05-01,352.857
251,2022-06-01,349.625


In [432]:
# Consumer Price Index
df_CPI = pd.read_csv("CPIAUCSL.csv", names = ["DATE", "CPI"], skiprows = 1).drop([252])
print(df_CPI.shape)
df_CPI.tail()

(252, 2)


,DATE,CPI
247,2022-02-01,284.182
248,2022-03-01,287.708
249,2022-04-01,288.663
250,2022-05-01,291.474
251,2022-06-01,295.328


In [433]:
# Monthly new house supply
df_house = pd.read_csv("Monthly_house_supply.csv", names = ["DATE", "Houses"], skiprows = 1).drop([252])
print(df_house.shape)
df_house.tail()

(252, 2)


,DATE,Houses
247,2022-02-01,6.0
248,2022-03-01,7.0
249,2022-04-01,8.4
250,2022-05-01,8.4
251,2022-06-01,9.2


In [434]:
# Population above 65

df_oldpop = pd.read_csv("oldpop.csv", names = ["DATE", "old_percent"], skiprows = 1)
print(df_oldpop.shape)
df_oldpop.tail()

(21, 2)


,DATE,old_percent
16,2017-01-01,15.419366
17,2018-01-01,15.807654
18,2019-01-01,16.209606
19,2020-01-01,16.630926
20,2021-01-01,17.037362


In [435]:
# Urban Population Percent

df_urban = pd.read_excel("urban_pop.ods", engine = "odf")
print(df_urban.shape)
df_urban.tail()

(22, 2)


,DATE,Urban_pop
17,2018-01-01,82.058
18,2019-01-01,82.256
19,2020-01-01,82.459
20,2021-01-01,82.664
21,2022-01-01,82.873


In [436]:
# Housing Subsidies

df_subsidy = pd.read_csv("Housing_Subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
print(df_subsidy.shape)
df_subsidy.tail()

(20, 2)


,DATE,Subsidy
15,2016-01-01,37.550
16,2017-01-01,35.771
17,2018-01-01,38.859
18,2019-01-01,40.185
19,2020-01-01,44.145


In [437]:
# Working age population

df_working = pd.read_csv("Working_age.csv", names = ["DATE", "Working_Population"], skiprows = 1).drop([252])
print(df_working.shape)
df_working.tail()

(252, 2)


,DATE,Working_Population
247,2022-02-01,207155867.2
248,2022-03-01,206979442.0
249,2022-04-01,207117362.0
250,2022-05-01,207356962.3
251,2022-06-01,207486228.4


In [438]:
# Real Median Household Income

df_income = pd.read_csv("Median_Income.csv", names = ["DATE", "Income"], skiprows = 1)
print(df_income.shape)
df_income.tail()

(20, 2)


,DATE,Income
15,2016-01-01,63683
16,2017-01-01,64557
17,2018-01-01,65127
18,2019-01-01,69560
19,2020-01-01,67521


In [439]:
# Number of households

df_households = pd.read_csv("Households.csv", names = ["DATE", "Num_Households"], skiprows = 1)
print(df_households.shape)
df_households.tail()

(20, 2)


,DATE,Num_Households
15,2016-01-01,125819.0
16,2017-01-01,126224.0
17,2018-01-01,127586.0
18,2019-01-01,128579.0
19,2020-01-01,128451.0


In [440]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
0,113.491,2001,7,46390.000000,181624734.6,4.2,177.4,4.6,143.4,3.77,79.057,108209.0,61889.0,20.573,12.313929
1,114.166,2001,8,46393.333333,181840987.1,4.4,177.4,4.9,143.3,3.65,79.057,108209.0,61889.0,20.573,12.313929
2,114.811,2001,9,46396.666667,182058514.8,4.4,178.1,5.0,143.0,3.07,79.057,108209.0,61889.0,20.573,12.313929
3,115.308,2001,10,46400.000000,182240152.0,4.3,177.6,5.3,142.3,2.49,79.057,108209.0,61889.0,20.573,12.313929
4,115.855,2001,11,46494.333333,182436924.9,4.1,177.5,5.5,142.1,2.09,79.057,108209.0,61889.0,20.573,12.313929


In [441]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()

(252, 10)


,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS
DATE,,,,,,,,,,
2001-07-01,113.491,2001,7,46390.0,181624734.6,4.2,177.4,4.6,143.4,3.77
2001-08-01,114.166,2001,8,NaN,181840987.1,4.4,177.4,4.9,143.3,3.65
2001-09-01,114.811,2001,9,NaN,182058514.8,4.4,178.1,5.0,143.0,3.07
2001-10-01,115.308,2001,10,46400.0,182240152.0,4.3,177.6,5.3,142.3,2.49
2001-11-01,115.855,2001,11,NaN,182436924.9,4.1,177.5,5.5,142.1,2.09


In [442]:
# Merging other dataframes 
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,
2001-07-01,113.491,2001,7,46390.0,181624734.6,4.2,177.4,4.6,143.4,3.77,79.057,108209.0,61889.0,20.573,12.313929
2001-08-01,114.166,2001,8,NaN,181840987.1,4.4,177.4,4.9,143.3,3.65,79.057,108209.0,61889.0,20.573,12.313929
2001-09-01,114.811,2001,9,NaN,182058514.8,4.4,178.1,5.0,143.0,3.07,79.057,108209.0,61889.0,20.573,12.313929
2001-10-01,115.308,2001,10,46400.0,182240152.0,4.3,177.6,5.3,142.3,2.49,79.057,108209.0,61889.0,20.573,12.313929
2001-11-01,115.855,2001,11,NaN,182436924.9,4.1,177.5,5.5,142.1,2.09,79.057,108209.0,61889.0,20.573,12.313929


In [443]:
df.isna().sum()

CSUSHPISA               0
Year                    0
Month                   0
Per_Capita_GDP        168
Working_Population      0
Houses                  0
CPI                     0
UNRATE                  0
Cons_Materials          0
FEDFUNDS                0
Urban_pop               0
Num_Households         18
Income                 18
Subsidy                18
old_percent             6
dtype: int64

The "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other columns is due to unavailability of fresh data. We will first fill the missing values in the "Per_Capita_GDP" column using linear interpolation. We will drop the rows having missing values in the other columns. This means that we will use data from 2001 to 2020

In [444]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [445]:
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,
2001-07-01,113.491,2001,7,46390.000000,181624734.6,4.2,177.4,4.6,143.4,3.77,79.057,108209.0,61889.0,20.573,12.313929
2001-08-01,114.166,2001,8,46393.333333,181840987.1,4.4,177.4,4.9,143.3,3.65,79.057,108209.0,61889.0,20.573,12.313929
2001-09-01,114.811,2001,9,46396.666667,182058514.8,4.4,178.1,5.0,143.0,3.07,79.057,108209.0,61889.0,20.573,12.313929
2001-10-01,115.308,2001,10,46400.000000,182240152.0,4.3,177.6,5.3,142.3,2.49,79.057,108209.0,61889.0,20.573,12.313929
2001-11-01,115.855,2001,11,46494.333333,182436924.9,4.1,177.5,5.5,142.1,2.09,79.057,108209.0,61889.0,20.573,12.313929


In [446]:
df.dropna(inplace = True)

In [447]:
df.isna().sum()

CSUSHPISA             0
Year                  0
Month                 0
Per_Capita_GDP        0
Working_Population    0
Houses                0
CPI                   0
UNRATE                0
Cons_Materials        0
FEDFUNDS              0
Urban_pop             0
Num_Households        0
Income                0
Subsidy               0
old_percent           0
dtype: int64

In [448]:
df.shape

(234, 15)

In [449]:
df.tail()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,
2020-08-01,222.507,2020,8,56133.000000,205827085.3,3.3,259.580,8.4,240.9,0.10,82.459,128451.0,67521.0,44.145,16.630926
2020-09-01,225.756,2020,9,56333.000000,206122572.1,3.4,260.190,7.9,246.9,0.09,82.459,128451.0,67521.0,44.145,16.630926
2020-10-01,229.409,2020,10,56533.000000,206068465.2,3.4,260.352,6.9,246.4,0.09,82.459,128451.0,67521.0,44.145,16.630926
2020-11-01,232.726,2020,11,56823.666667,206125599.5,4.1,260.721,6.7,244.3,0.09,82.459,128451.0,67521.0,44.145,16.630926
2020-12-01,235.776,2020,12,57114.333333,205988747.7,4.1,261.564,6.7,248.0,0.09,82.459,128451.0,67521.0,44.145,16.630926


This is our preprocessed datset. Let's save it as "prepared_dataset.csv".

In [450]:
df.to_csv("prepared_dataset.csv")

In [452]:
df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,
2001-07-01,113.491,2001,7,46390.000000,181624734.6,4.2,177.4,4.6,143.4,3.77,79.057,108209.0,61889.0,20.573,12.313929
2001-08-01,114.166,2001,8,46393.333333,181840987.1,4.4,177.4,4.9,143.3,3.65,79.057,108209.0,61889.0,20.573,12.313929
2001-09-01,114.811,2001,9,46396.666667,182058514.8,4.4,178.1,5.0,143.0,3.07,79.057,108209.0,61889.0,20.573,12.313929
2001-10-01,115.308,2001,10,46400.000000,182240152.0,4.3,177.6,5.3,142.3,2.49,79.057,108209.0,61889.0,20.573,12.313929
2001-11-01,115.855,2001,11,46494.333333,182436924.9,4.1,177.5,5.5,142.1,2.09,79.057,108209.0,61889.0,20.573,12.313929
